# WOFOST crop database

There is a GitHub repository containing calibtated parameters for a range of varieties of crops. These parameters were used, e.g., in the Hupselbrook test case provided with the SWAP model for potato variety. As these are reusable, it made sense to tap into that database to make creation of valid crop files for pySWAP easier.

In this notebook, we will create a .crp file for potato variety used in the hupselbrook test case.

In [ ]:
import pyswap as psp

psp.setup_logging(level="INFO")

## Exploring the database

pySWAP contains a simple wrapper around A. Wit's crop database. We can use built in classes to get an overview of available data. Then we can check which crop types are available in the database.

In [ ]:
cropdb = psp.db.WOFOSTCropDB()
cropdb.croptypes

When we call `load_crop_file` on the database object with the name of a specific crop type, we will get a WOFOSTCropFile object. We can then inspect that object to see the file's metadata, parameters in individual sections, available ecotypes and varieties.

In [ ]:
cropfile = cropdb.load_crop_file("rice")

In [ ]:
cropfile.metadata

In [ ]:
print(cropfile.ecotypes)
print(cropfile.varieties)

In [ ]:
rice_501 = cropfile.get_variety("Rice_501")

The new object is `CropVariety` object. From here you can view the metadata of the particular variety or obtain just the parameters as a dictionary. Mind that at this stage, all tables are just arrays of numbers, the same way as it is formatted in the .yaml files. pySWAP automatically converts them in tables when they are used to populate the parameters of crop settings classes.

## Using the CropVariety object to populate the crop settings class

The point of having this database integration is to enable populating some of the wofost crop parameters used by swap directly from the files. A lot of the parameters we still have to set manually, namely Preparation, ScheduledIrrigation, Interception, OxygenStress, DroughtStress, etc.

In [ ]:
potato_prep = psp.components.crop.Preparation(
    swprep=0,
    swsow=0,
    swgerm=2,
    tsumemeopt=170.0,
    tbasem=3.0,
    teffmx=18.0,
    hdrygerm=-500.0,
    hwetgerm=-100.0,
    zgerm=-10.0,
    agerm=203.0,
    dvsend=2.0,
    swharv=0,
)

scheduled_irrigation = psp.components.irrigation.ScheduledIrrigation(schedule=0)
interception = psp.components.crop.Interception(swinter=1, cofab=0.25)

potato_ox_stress = psp.components.crop.OxygenStress(
    swoxygen=1,
    swwrtnonox=1,
    aeratecrit=0.5,
    hlim1=-10.0,
    hlim2u=-25.0,
    hlim2l=-25.0,
    swrootradius=2,
    root_radiuso2=0.00015,
)

potato_dr_stress = psp.components.crop.DroughtStress(
    swdrought=1,
    hlim3h=-300.0,
    hlim3l=-500.0,
    hlim4=-10000.0,
    adcrh=0.5,
    adcrl=0.1,
)

Then, we can load the potato_701 variety from the WOFOST database and create the CropDevelopmentSettings object like so:

In [ ]:
db = psp.db.WOFOSTCropDB()
potato = db.load_crop_file("potato")
potato_params = potato.get_variety("Potato_701")

potato_cropdev_settings = psp.components.crop.CropDevelopmentSettingsWOFOST(
    wofost_variety=potato_params,
    swcf=2,
    albedo=0.19,
    laiem=0.0589,
    ssa=0.0,
    kdif=1.0,
    rsc=207.0,
    rsw=0.0,
    kdir=0.75,
    eff=0.45,
    swrd=2,
    rdc=50.0,
    swdmi2rd=1,
)

# By calling the `update_from_wofost method`, the CropDevelopmentSettingsWOFOST object
# will be updated with the values from the WOFOST crop file and the model will be evaluated,
# to make sure that some of the required settings are not missing.
potato_cropdev_settings.update_from_wofost()

## Additional tables defined in a yaml file (new since v0.2.9)
To complement the additional tables that are not normally found in the wofost database from another file, we can use update_from_yaml method passing the yaml path. The filr has to be in the following format:

```yaml
Version: 1.0.0
Metadata: &meta
  Creator:
  Contributor:
  # Any additional metadata fields you want
PredefinedArrays:
#   DVS: &dws [0.0, 0.3, 0.5, 0.7, 1.0, 1.4, 2.0] <-- some arrays you want to reuse
CropParameters: # CropParameters and SWAPInput sections are essential.
  SWAPInput:
    # in contrast to the WOFOST database, here the columns are defined as a dictionary of lists. JSON resulting from this yaml structure looks like this: {CHTB: {DVS: [values...], CH: [values]}}
    CHTB:
      DVS: [0.0, 1.0, 2.0] 
      CH: [1.0, 40.0, 50.0]
    RDCTB:
      RRD: [0.0, 1.0]
      RDENS: [1.0, 0.0]
```

In [ ]:
from pyswap.testcase import get_path

# getting test dataset
potato_tables = get_path("hupselbrook", "potato_tables_yaml")

Further we proceed with creation of the `CropFile` object, which goes directly to the `Model`. You can observe the content of the crop file by calling `CropFile.crp` property.

In [ ]:
potato_cropdev_settings.update_from_yaml(potato_tables)

Varify that the chtb and rdctb are added despite we did not define them in the code.

In [ ]:
print(potato_cropdev_settings.chtb)
print(potato_cropdev_settings.rdctb)

In [ ]:
crppotato = psp.components.crop.CropFile(
    name="potatod",
    prep=potato_prep,
    cropdev_settings=potato_cropdev_settings,
    oxygenstress=potato_ox_stress,
    droughtstress=potato_dr_stress,
    interception=interception,
    scheduledirrigation=scheduled_irrigation,
)

In [ ]:
print(crppotato.crp)

## Changing the values (for example during model calibration)
The WOFOST database integration allows you draw calibrated settings from the database into pySWAP. If you wish to change some of the settings, you should do it by updating the base pySWAP classes (e.g., CropDevalopmentSettings), like so:

In [ ]:
potato_cropdev_settings_tsum1_up = potato_cropdev_settings.update({"tsum1": 900})

In [ ]:
crppotato_tsum1_up = crppotato.update({
    "cropdev_settings": potato_cropdev_settings_tsum1_up
})

In [ ]:
print(crppotato_tsum1_up.cropdev_settings.tsum1)